# IBM Data Science Coursera Capstone Project Jupyter Notebook

Author: Mark Hanson

## Background
This Jupyter notebook will be used for the final capstone project in the Coursera IBM Data Science Professional class.  In brief the requirements are to leverage Foursquare location data to explore or compare any chosen neighborhoods or cities or to come up with a problem that can be solved with data obtained with the Foursquare API. Unfortunately, Foursquare time-series data is only available to verified managers of Foursquare venues which makes answering some more interesting questions more difficult. In addition, it would seem that Google would have a much more rich dataset which as mentioned in the course is no longer available at a reasonable price. Despite these factors, I have come up with what I think is an interesting project.

#### Description of my capstone project
For my capstone project I have chosen to cluster the cities in the Minneapolis / Saint Paul 7 county metropolitan area similar to what was done in the labs. However, instead of clustering based on similar venues I have chosen to use home valuation data from Zillow to cluster zip codes in the area by similar home values and then add Foursquare venue data for each zip code.

## Introduction / Business Problem
The Minneapolis-St.Paul (MSP) Metropolitan Area is home to a significant number of both public and private companies including several on the Fortune 500 list. Some notable companies with headquarters or major operations in the area include United Health, Target, Best Buy, CHS, 3M, US Bank, Supervalue, General Mills, C.H. Robinson, Ecolab, Land O Lakes, Ameriprise Financial, Xcel Energy, Thrivent Financial, Securian Financial, Well Fargo, Cargill, Hormel, Medtronic, Honeywell, and Patterson. With such a large and diverse set of large companies in the Minneapolis-St.Paul area, there are bound to be employees that accept a new position within their respective company and move to the region.  Being new to the area and unfamiliar with the different cities that comprise the MSP surrounding suburbs makes finding a place to buy a home challenging.  Realtors can help with this as they know the area.  But many of the transplanted workers may be data driven and what a more rigorous characterization of the various suburbs.  Many factors will come into play when selecting a place to consider buying a home.  Price range is usually one of the chief considerations with commute time, school district, and venues and attractions also major factors.

To help both realtors and prospective buyers better understand various suburbs and cities within the Minneapolis-St.Paul 7 county metro area, I have acquired and summarized Zillow data on median home values by zip code.  I used a k-mean clustering algorithm to group the various zip codes into 10 different price ranges and created a maps showing where the various price ranges fall geographically.  To provide additional information I calculated the 5, 10, 15, and 20 year compound annual growth rate in the median home value for each zip code.  Finally, I used Foursquare to add the top venues in each zip code.

During the analysis clustering was performed both with and without the venue data.  Clustering with venue data resulted in very little differentiation between the clusters and overly broad home price ranges.  Since price point is usually the over-arching concern when buying a home, the decision to cluster just on median home price and then add venue data seems appropriate and best solves the business problem.

## Data
Zillow is an online real estate database company that has data sets of median home values by zip code for download. Zillow data for zip codes in Minnesota and Wisconsin was downloaded and a subset of it used to cluster zip codes in the 7 county Minneapolis-St. Paul Metro area into price ranges by median home value.  Other data sets used in this analysis include zip code centroid geo-coordinates data from CivicSpace Labs, zip code shape geoJSON data from the US Census Bureau which was processed and posted to GitHub by OpenData.  Additional analysis will include computing the compound annual growth rate over 5, 10, 15, and 20 year periods as well as providing the top 10 venues in each zip code using Foursquare data.

Data attribution (sources):
1. Home value data attribution: “Data acquired from Zillow.com/data on April 10, 2019. Aggregated data on this page is made freely available by Zillow for non-commercial use.”
2. Zip code centroid data attribution:
    - Dataset Identifier: us-zip-code-latitude-and-longitude
    - License: Creative Commons Attribution-ShareAlike
    - Modified: February 9, 2018 10:31 AM
    - Publisher: CivicSpace Labs
    - Reference: https://boutell.com/zipcodes/
    - Attributions: Copyright 2004 CivicSpace Labs
    - Last processing:
        - February 9, 2018 10:39 AM (metadata)
        - February 9, 2018 10:39 AM (data)
3. Zip code GeoJSON attributions:
    - https://www.census.gov/cgi-bin/geo/shapefiles2010/layers.cgi
    - https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/README.md
4. Foursquare developer API attribution:
    - https://foursquare.com/developers/apps

#### Install and import packages

In [1]:
# Install geocoder package with conda
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#print('geopy installed!')

In [2]:
# install folium
#!conda install -c conda-forge folium=0.5.0 --yes
#print('folium installed!')

In [3]:
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [1]:
# import other modules
import pandas as pd
from pandas.io.json import json_normalize

import numpy as np
import requests
import json
import bs4
from bs4 import BeautifulSoup as bs

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [2]:
import folium

#### Import Zillow data

1. Home value data attribution: “Data acquired from Zillow.com/data on April 10, 2019. Aggregated data on this page is made freely available by Zillow for non-commercial use.”

NOTE: While working on project, skip down to section MP 3 and import zill_MSP_data.csv or zill_MSP_data_test.csv instead of recreating entire df.

Create a new dataframe with Minneapolis / St. Paul 7 county metro area, drop unused columns, rename ZipCode, and reset index

In [3]:
zill_MNWI_df = pd.read_csv('Zillow_Home_Value_Index_MN_WI.csv')
zill_MN_df = zill_MNWI_df[zill_MNWI_df['State']=='MN']
print(zill_MN_df.shape)
#zill_MN_df.head()

(413, 282)


In [4]:
zill_MN_df = zill_MN_df.drop(['RegionID'], axis=1)
#zill_MN_df = zill_MN_df.drop(zill_MNWI_df.columns.to_series()['1996-04':'2013-12'], axis=1)
zill_MN_df = zill_MN_df.rename(index=str, columns={'RegionName':'ZipCode'})
zill_MN_df.reset_index(drop=True, inplace=True)
#zill_MN_df.head()

In [5]:
zill_MSP_df = zill_MN_df[zill_MN_df['CountyName'].isin(['Anoka County','Carver County','Dakota County','Hennepin County','Ramsey County','Scott County','Washington County'])]

In [6]:
print(zill_MSP_df.shape)
zill_MSP_df.head()

(142, 281)


,ZipCode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02
1,55124,Apple Valley,MN,Minneapolis-St. Paul-Bloomington,Dakota County,472,118900.0,119400.0,119900.0,120600.0,...,266300,267600,268100,268800,269800,271000,272300,273900,275600,276800
3,55044,Lakeville,MN,Minneapolis-St. Paul-Bloomington,Dakota County,779,136600.0,136500.0,137000.0,138200.0,...,342800,343900,344300,344400,344700,345900,347400,349000,350600,352000
4,55337,Burnsville,MN,Minneapolis-St. Paul-Bloomington,Dakota County,806,119800.0,120100.0,120400.0,120700.0,...,262700,263800,264400,265400,266300,267300,268600,270300,272100,273500
5,55106,Saint Paul,MN,Minneapolis-St. Paul-Bloomington,Ramsey County,841,70800.0,70800.0,70900.0,71000.0,...,179900,179900,180100,180100,180000,180700,182200,183600,184700,185600
6,55303,Ramsey,MN,Minneapolis-St. Paul-Bloomington,Anoka County,850,94800.0,95100.0,95500.0,95800.0,...,243200,244200,244800,246400,248500,249800,250800,252100,253900,255200


#### Import Zip Code Centroid Geocoordinates

Notes about geocoordinates:

Originally looped through all the cities in the MSP dataframe and added the latitude and longitude data to the new df using geolocator.  But geolocator uses city and state to get latitude and longitude which results in the same information for all the zip codes in a larger city.  For example, using geolocator results in all 12 zip codes in St. Paul having the same geolocation. Foursquare uses that geolocation to get venue data so all of St. Paul would have the same venues regardless of zip code.  Zillow home value data has data for each zip code.  This mismatch would have obscured the relationship between home values (by zip code) and venues (by city) resulting in poor quality data.  By getting geolocations for each zip code on-line the mismatch can be resolved and each zip code can have an associated list of venues around it.

Found dataset on OpenDataSoft (Civic Space Labs) that links zip code to single geolocation (not boundries as with geoJSON). OpenDataSoft has an API to get data which I used but could only get 100 records which is likely due to a quota limit. Since data is available for download as csv anyway I used a csv to get zipcode centroid data.

2. Zip code centroid data attribution:
    - Dataset Identifier: us-zip-code-latitude-and-longitude
    - License: Creative Commons Attribution-ShareAlike
    - Modified: February 9, 2018 10:31 AM
    - Publisher: CivicSpace Labs
    - Reference: https://boutell.com/zipcodes/
    - Attributions: Copyright 2004 CivicSpace Labs
    - Last processing:
        - February 9, 2018 10:39 AM (metadata)
        - February 9, 2018 10:39 AM (data)

In [7]:
# get geocoordinates for a point in each zipcode
MSP_ZipCode_Geo = pd.read_csv('us-zip-code-latitude-and-longitude.csv', sep=';')
MSP_ZipCode_Geo.drop(columns=['Timezone','Daylight savings time flag', 'geopoint'], inplace=True)
MSP_ZipCode_Geo = MSP_ZipCode_Geo[MSP_ZipCode_Geo['State'].isin(['MN'])]
MSP_ZipCode_Geo.rename(columns={'Zip':'ZipCode'}, inplace=True)
MSP_ZipCode_Geo[MSP_ZipCode_Geo['ZipCode'].isin(zill_MSP_df['ZipCode'])].shape

(141, 5)

Merge MSP_ZipCode_Geo df into zill_MSP_df and reoder columns so latitude and longitude are after city and state.

In [8]:
# merge geocoordinates by zipcode into home value df
zill_MSP_df = pd.merge(zill_MSP_df, MSP_ZipCode_Geo, on=['ZipCode'])

In [9]:
# drop city and state from MSP_ZipCode_Geo as they are less specific that those from zill_MSP_df
zill_MSP_df.drop(columns=['City_y','State_y'], inplace=True)
zill_MSP_df.rename(columns={'City_x' : 'City', 'State_x' : 'State'}, inplace=True)

In [10]:
# move geocoordinates from end columns to right after city name
cols = zill_MSP_df.columns.tolist()
cols = cols[0:2] + cols[-2:] + cols[2:-2]
zill_MSP_df = zill_MSP_df[cols]

In [11]:
print(zill_MSP_df.shape)
zill_MSP_df.head()

(141, 283)


,ZipCode,City,Latitude,Longitude,State,Metro,CountyName,SizeRank,1996-04,1996-05,...,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02
0,55124,Apple Valley,44.743963,-93.20624,MN,Minneapolis-St. Paul-Bloomington,Dakota County,472,118900.0,119400.0,...,266300,267600,268100,268800,269800,271000,272300,273900,275600,276800
1,55044,Lakeville,44.669564,-93.26654,MN,Minneapolis-St. Paul-Bloomington,Dakota County,779,136600.0,136500.0,...,342800,343900,344300,344400,344700,345900,347400,349000,350600,352000
2,55337,Burnsville,44.770297,-93.27302,MN,Minneapolis-St. Paul-Bloomington,Dakota County,806,119800.0,120100.0,...,262700,263800,264400,265400,266300,267300,268600,270300,272100,273500
3,55106,Saint Paul,44.967565,-93.05001,MN,Minneapolis-St. Paul-Bloomington,Ramsey County,841,70800.0,70800.0,...,179900,179900,180100,180100,180000,180700,182200,183600,184700,185600
4,55303,Ramsey,45.247509,-93.41800,MN,Minneapolis-St. Paul-Bloomington,Anoka County,850,94800.0,95100.0,...,243200,244200,244800,246400,248500,249800,250800,252100,253900,255200


In [24]:
# write zill_MSP_df to csv so don't have to recreate it everytime using slow geolocator
#zill_MSP_df.to_csv(path_or_buf='zill_MSP_data.csv')

#### Load cleaned data set from CSV while working on project
While working on project, import zill_MSP_data.csv to df instead of re-creating it each time.

In [12]:
# read zill_MSP_df from csv while working on project
zill_MSP_df = pd.read_csv('zill_MSP_data.csv')
zill_MSP_df = zill_MSP_df.drop(columns=['Unnamed: 0'])
print(zill_MSP_df.shape)
zill_MSP_df.head()

(141, 283)


,ZipCode,City,Latitude,Longitude,State,Metro,CountyName,SizeRank,1996-04,1996-05,...,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02
0,55124,Apple Valley,44.743963,-93.20624,MN,Minneapolis-St. Paul-Bloomington,Dakota County,472,118900.0,119400.0,...,266300,267600,268100,268800,269800,271000,272300,273900,275600,276800
1,55044,Lakeville,44.669564,-93.26654,MN,Minneapolis-St. Paul-Bloomington,Dakota County,779,136600.0,136500.0,...,342800,343900,344300,344400,344700,345900,347400,349000,350600,352000
2,55337,Burnsville,44.770297,-93.27302,MN,Minneapolis-St. Paul-Bloomington,Dakota County,806,119800.0,120100.0,...,262700,263800,264400,265400,266300,267300,268600,270300,272100,273500
3,55106,Saint Paul,44.967565,-93.05001,MN,Minneapolis-St. Paul-Bloomington,Ramsey County,841,70800.0,70800.0,...,179900,179900,180100,180100,180000,180700,182200,183600,184700,185600
4,55303,Ramsey,45.247509,-93.41800,MN,Minneapolis-St. Paul-Bloomington,Anoka County,850,94800.0,95100.0,...,243200,244200,244800,246400,248500,249800,250800,252100,253900,255200


#### Cluster the data by the most recent median home value
 - Create a summary data frame of ZipCode, City, Lat, Lng, State, Metro, CountyName, SizeRank, 2019-02 that will be built up through each stage as more summary data is obtained
 - Create a normalized clustering data frame
 - Cluster data by normalized most recent home value and insert cluster labels into summary data frame
 - Create combined vertical box plot of clusters and their associated median home value
 - Create choropleth map of median home value by zip code

Create a summary data frame of ZipCode, City, Lat, Lng, State, Metro, CountyName, SizeRank, 2019-02 that will be built up through each stage as more summary data is obtained

In [13]:
# create a summary data frame
MSP_summary_df = zill_MSP_df.copy(deep=True)
MSP_summary_df = MSP_summary_df.drop(columns=MSP_summary_df.columns.to_series()['1996-04':'2019-01'], axis=1)
MSP_summary_df.head()

,ZipCode,City,Latitude,Longitude,State,Metro,CountyName,SizeRank,2019-02
0,55124,Apple Valley,44.743963,-93.20624,MN,Minneapolis-St. Paul-Bloomington,Dakota County,472,276800
1,55044,Lakeville,44.669564,-93.26654,MN,Minneapolis-St. Paul-Bloomington,Dakota County,779,352000
2,55337,Burnsville,44.770297,-93.27302,MN,Minneapolis-St. Paul-Bloomington,Dakota County,806,273500
3,55106,Saint Paul,44.967565,-93.05001,MN,Minneapolis-St. Paul-Bloomington,Ramsey County,841,185600
4,55303,Ramsey,45.247509,-93.41800,MN,Minneapolis-St. Paul-Bloomington,Anoka County,850,255200


#### Create a data frame for clustering and evaluate k- means accuracy for various cluster sizes
 - Create a data frame of just median home value for clustering
 - Normalize clustering data frame
 - Evaluate and graph accuracy - as measured by inertia - with various values of k (number of clusters) and select the k value at or around the knee point.

In [14]:
# create a data frame of just median home value
MSP_med_HV = MSP_summary_df.copy(deep=True)
MSP_med_HV = MSP_med_HV.drop(['ZipCode','City','Latitude','Longitude','State','Metro','CountyName','SizeRank'], axis=1)
#MSP_med_HV.head()

In [15]:
# create df with normalized home values in range
scaler = RobustScaler()
MSP_med_HV.iloc[:,0:1] = scaler.fit_transform(MSP_med_HV['2019-02'].values.reshape(-1,1))
MSP_med_HV.head()

,2019-02
0,-0.052280
1,0.784205
2,-0.088988
3,-1.066741
4,-0.292547


Insert cluster labels into summary data frame

#### Get venue data by zip code and add it to data frame
 - Cluster the zip code by the various compound annual growth rates and plot as choropleth maps

#### Explore Cities in the 7 county Minneapolis-St.Paul metropolitan area
As in labs, define a function to get FourSquare venue data for all the cities in the Minneapolis-St.Paul metro area. Originally I had issues with rural locations having no venues (too small a radius) and urban locations being clustered together due to too large a radius so experimented with radius and eliminating rural areas by limiting to 7 county metro area. Ultimately selected a 1.61km (1mi) radius around each zip code geolocation and ran query during business hours. Also determined that overly general clusters problem was alleviated by normailzing home values to a wider range of values.

In [28]:
# Four square credentials

In [29]:
# set radius to 1610m (approx 1 mi) and limit search to 100 venues;
def getNearbyVenues(zipcode, names, latitudes, longitudes):
    radius = 1610
    LIMIT = 100   
    
    venues_list=[]
    for zipcode, name, lat, lng in zip(zipcode, names, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # deal with case where rural zipcodes are bigger and venues might not be found within 2km so increase to 5km
#        if results == []:
#            radius = 1610
        
        # return only relevant information for each nearby venue                
        venues_list.append([(
            zipcode,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ZipCode','City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each zip code and create a new dataframe called MSP_venues.

In [30]:
MSP_venues = getNearbyVenues(zipcode = zill_MSP_df['ZipCode'], names = zill_MSP_df['City'], latitudes = zill_MSP_df['Latitude'], longitudes = zill_MSP_df['Longitude'])

In [33]:
print('There are {} uniques categories.'.format(len(MSP_venues['Venue Category'].unique())))

There are 370 uniques categories.


In [34]:
# write MSP_venues df to csv so don't have to recreate it everytime also time dependent last run and saved 12:33 PM 5/1/19
MSP_venues.to_csv(path_or_buf='MSP_venues_data.csv')

While working on project, import MSP_venues.csv to df instead of re-creating it each time.

In [32]:
# read MSP_venues df from csv so don't have to run as many queries against Foursquare
MSP_venues = pd.read_csv('MSP_venues_data.csv')
MSP_venues = MSP_venues.drop(columns=['Unnamed: 0'])
print(MSP_venues.shape)
MSP_venues.head()

(5187, 8)


,ZipCode,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,55124,Apple Valley,44.743963,-93.20624,Five Guys,44.735794,-93.216446,Burger Joint
1,55124,Apple Valley,44.743963,-93.20624,Satay 2 Go,44.731270,-93.201148,Asian Restaurant
2,55124,Apple Valley,44.743963,-93.20624,Kwik Trip,44.739600,-93.218739,Convenience Store
3,55124,Apple Valley,44.743963,-93.20624,Cole's Salon,44.731770,-93.216405,Salon / Barbershop
4,55124,Apple Valley,44.743963,-93.20624,Jimmy John's,44.733264,-93.206591,Sandwich Place


#### Analyze Each Neighborhood

In [35]:
# one hot encoding
MSP_onehot = pd.get_dummies(MSP_venues[['Venue Category']], prefix="", prefix_sep="")

# add zipcode column back to dataframe
MSP_onehot['ZipCode'] = MSP_venues['ZipCode'] 

# move zipcode column to the first column
fixed_columns = [MSP_onehot.columns[-1]] + list(MSP_onehot.columns[:-1])
MSP_onehot = MSP_onehot[fixed_columns]

#MSP_onehot.head()

In [36]:
MSP_onehot.shape

(5187, 371)

In [37]:
# list the average number of venues of each type in each zipcode
MSP_grouped = MSP_onehot.groupby('ZipCode').mean().reset_index()
MSP_grouped.head()

,ZipCode,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Service,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,55001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,55005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,55011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,55014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe zipcode_venues_sorted to list the top 10 venues for each neighborhood.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ZipCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zipcode_venues_sorted = pd.DataFrame(columns=columns)
zipcode_venues_sorted['ZipCode'] = MSP_grouped['ZipCode']

for ind in np.arange(MSP_grouped.shape[0]):
    zipcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(MSP_grouped.iloc[ind, :], num_top_venues)

print(zipcode_venues_sorted.shape)
zipcode_venues_sorted.head()

(141, 11)


,ZipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,55001,Athletics & Sports,Zoo Exhibit,Dry Cleaner,Electronics Store,Elementary School,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit
1,55003,Breakfast Spot,Video Store,Athletics & Sports,New American Restaurant,Bar,BBQ Joint,Boat or Ferry,Flea Market,Exhibit,English Restaurant
2,55005,Sandwich Place,Business Service,Gas Station,Zoo Exhibit,Dry Cleaner,Electronics Store,Elementary School,English Restaurant,Entertainment Service,Ethiopian Restaurant
3,55011,Hardware Store,Forest,Zoo Exhibit,Electronics Store,Elementary School,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit
4,55014,American Restaurant,Pizza Place,Baseball Field,Fast Food Restaurant,Video Store,Hockey Arena,Bar,Thrift / Vintage Store,Big Box Store,Playground


Add 10 most common venues for each zip code to MSP_summary_df

In [40]:
# add 10 most common venues for each zip code to MSP_summary_df
MSP_summary_df = MSP_summary_df.join(zipcode_venues_sorted.set_index('ZipCode'), on='ZipCode')
MSP_summary_df.head()

,ClusterLabel,ZipCode,City,Latitude,Longitude,State,Metro,CountyName,SizeRank,2019-02,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,8,55124,Apple Valley,44.743963,-93.20624,MN,Minneapolis-St. Paul-Bloomington,Dakota County,472,276800,...,Park,Coffee Shop,Asian Restaurant,Soccer Field,Sandwich Place,Gym / Fitness Center,Convenience Store,Pizza Place,Mexican Restaurant,American Restaurant
1,9,55044,Lakeville,44.669564,-93.26654,MN,Minneapolis-St. Paul-Bloomington,Dakota County,779,352000,...,Gym / Fitness Center,Golf Course,Home Service,Lake,Beach,Skating Rink,Department Store,Harbor / Marina,Disc Golf,Zoo Exhibit
2,8,55337,Burnsville,44.770297,-93.27302,MN,Minneapolis-St. Paul-Bloomington,Dakota County,806,273500,...,Coffee Shop,Pizza Place,Hotel,Sandwich Place,Salon / Barbershop,American Restaurant,Video Store,Park,Smoke Shop,Skating Rink
3,7,55106,Saint Paul,44.967565,-93.05001,MN,Minneapolis-St. Paul-Bloomington,Ramsey County,841,185600,...,Mexican Restaurant,Grocery Store,Park,Convenience Store,Sandwich Place,Gym,Gym / Fitness Center,Pharmacy,Pizza Place,Fast Food Restaurant
4,0,55303,Ramsey,45.247509,-93.41800,MN,Minneapolis-St. Paul-Bloomington,Anoka County,850,255200,...,Campground,Convenience Store,Playground,Dog Run,Big Box Store,Farm,Elementary School,English Restaurant,Entertainment Service,Ethiopian Restaurant


Get MN geoJSON data and create subset for Minneapolis-St.Paul Metro
 - Import zipcode geojson data for MN from github which used us census data:   
     3. Zip code GeoJSON attributions:
        - https://www.census.gov/cgi-bin/geo/shapefiles2010/layers.cgi
        - https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/README.md

In [128]:
mn_zipcode_file = open('mn_minnesota_zip_codes_geo.json')
mn_zipcode_str = mn_zipcode_file.read()
mn_zipcode_data = json.loads(mn_zipcode_str)

Create a subset of MN geoJSON for just Minneapolis-St.Paul metro zipcodes

In [129]:
# loop through items in state wide zipcode geoJSON file and get just 7 county metro area zipcodes (those in MSP_merged) and write to new geoJSON file
elem = 0 # use to count how many elements
MSP_geoJSON = {'type': 'FeatureCollection', 'features' : []} # dict to store matches; Note had { ... 'features' : dict()} which made features keys vs items as they are now with []

# loop through all the elements in the larger JSON file (which is now stored as a python dict); there is an element for each zipcode in MN
for element in mn_zipcode_data['features']:
    # loop through each zipcode element and look for the property where the zipcode is stored
    for k, v in element['properties'].items():
        if k == 'ZCTA5CE10':
            # loop through each zipcode in MSP_merged dataset
            for postcode in MSP_summary_df['ZipCode']:
                # if the current value from the larger JSON file matches it a zipcode in MSP_merged add the element to MN_geoJSON dict and incr elem
                if str(v) == str(postcode):
                    MSP_geoJSON['features'].append(mn_zipcode_data['features'][elem])  # flaw in logic!! [elem] should be mn_zipcode_data['features'][0 or whatever item number match found!]
            elem += 1    # increment the element count to keep track of how many elements

In [96]:
# write MSP_geoJSON to msp_zipcode_geo.json file
#with open('msp_zipcode_geo.json', 'w') as oufile:
#    json.dump(MSP_geoJSON, oufile)

In [130]:
# read MSP_geoJSON file
msp_zipcode_geo = r'msp_zipcode_geo.json'
#mn_zipcode_geo = r'mn_minnesota_zip_codes_geo.json'

Troubleshooting: Define a function to compare lists to find the zipcodes that are in list a but not in list b

In [236]:
def compare_lists(list_a, list_b):
    missing_in_b = []
    for x in list_a:
        if x not in list_b:
            missing_in_b.append(x)
#        print('zill: ', x)
    return missing_in_b

In [258]:
zill_ZipCode_List = zill_MSP_df['ZipCode'].tolist()
len(zill_ZipCode_List)

141

In [262]:
city_venues_sorted_ZipCode_List = city_venues_sorted['ZipCode'].tolist()
len(city_venues_sorted_ZipCode_List)

141

In [263]:
compare_lists(zill_ZipCode_List, city_venues_sorted_ZipCode_List)

[]

Print each zipcode along with the top 5 most common venues

In [303]:
#num_top_venues = 5

#for zipcode in MSP_grouped['ZipCode']:
#    print("----"+str(zipcode)+"----")
#    temp = MSP_grouped[MSP_grouped['ZipCode'] == zipcode].T.reset_index()   # .T transposes dataframe
#    temp.columns = ['venue','freq']
#    temp = temp.iloc[1:]
#    temp['freq'] = temp['freq'].astype(float)
#    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

### geoJSON visual section

In [348]:
# create a new dataframe as a subset of MSP_merged. Important change: Converted Zipcode from int64 to string since geoJSON has it stored as a string!
MSP_value_data = MSP_merged[['ZipCode','Feb-2019 Median Home Value']]
MSP_value_data = MSP_value_data.rename(columns={'Feb-2019 Median Home Value':'Value'})
MSP_value_data['ZipCode'] = MSP_value_data['ZipCode'].apply(str)
#MSP_value_data.dtypes

import zipcode geojson data for MN from this site; use to make choropleth maps etc.; https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/README.md

In [112]:
mn_zipcode_file = open('mn_minnesota_zip_codes_geo.json')
mn_zipcode_str = mn_zipcode_file.read()
mn_zipcode_data = json.loads(mn_zipcode_str)

Write smaller geojson for just 7 county area from full MN zipcode geojson

In [160]:
# loop through items in state wide zipcode geoJSON file and get just 7 county metro area zipcodes (those in MSP_merged) and write to new geoJSON file
elem = 0 # use to count how many elements
MSP_geoJSON = {'type': 'FeatureCollection', 'features' : []} # dict to store matches; Note had { ... 'features' : dict()} which made features keys vs items as they are now with []

# loop through all the elements in the larger JSON file (which is now stored as a python dict); there is an element for each zipcode in MN
for element in mn_zipcode_data['features']:
    # loop through each zipcode element and look for the property where the zipcode is stored
    for k, v in element['properties'].items():
        if k == 'ZCTA5CE10':
            # loop through each zipcode in MSP_merged dataset
            for postcode in MSP_merged['ZipCode']:
                # if the current value from the larger JSON file matches it a zipcode in MSP_merged add the element to MN_geoJSON dict and incr elem
                if str(v) == str(postcode):
                    MSP_geoJSON['features'].append(mn_zipcode_data['features'][elem])  # flaw in logic!! [elem] should be mn_zipcode_data['features'][0 or whatever item number match found!]
            elem += 1    # increment the element count to keep track of how many elements

In [161]:
# write MSP_geoJSON to msp_zipcode_geo.json file
#with open('msp_zipcode_geo.json', 'w') as oufile:
#    json.dump(MSP_geoJSON, oufile)

In [199]:
#MSP_geoJSON
msp_zipcode_geo = r'msp_zipcode_geo.json'
#mn_zipcode_geo = r'mn_minnesota_zip_codes_geo.json'